In [25]:
from guidance import models, instruction, gen
import guidance

# load the mistral model
MODEL_PATH = "/Users/nyeung/Projects/llama.cpp/models/mistral-7b-instruct-v0.2.Q4_K_M.gguf"
mistral = models.LlamaCpp(MODEL_PATH, n_gpu_layers=-1, n_context=2048)

# guidance function to generate task
@guidance
def generate_daily_plan(lm, persona, num_tasks=3):
    lm += f'''Generate {num_tasks} specific tasks that a {persona} does in a day: \n'''
    for i in range(3):
        lm += f'''Task {i+1}: "{gen(stop='"', name="tasks", temperature=1.0, list_append=True, max_tokens=100)}"\n'''
    return lm

@guidance
def estimate_duration(lm, persona, tasks):
    lm += "Estimate a realistic duration, in hours, of how much time a {persona} would take for each task: \n"
    for i in range(len(tasks)):
        lm += f'''Task {i+1} will take {persona} {gen(stop='"', regex="[0-9]", name="duration", temperature=0.7, max_tokens=10, list_append=True)} hours\n'''
    return lm

@guidance
def generate_start_times(lm, persona, tasks):
    lm += f"Generate a start time for when {persona} will start each task: \n"
    for i in range(len(tasks)):
        lm += f'''Task {i+1} will start at {gen(stop='"', regex="[0-9]:[0-9][0-9]", name="start_time", temperature=0.7, max_tokens=10, list_append=True)} hours\n'''
    return lm

In [11]:
import json

def load_persona(file_path):
    try:
        with open(file_path, 'r') as file:
            persona_data = json.load(file)
        persona = json.dumps(persona_data["persona"], indent=2)
        return f"""
        You are {persona['name']}. Your background is {persona['background']}. You like {persona['']}        
"""
    except FileNotFoundError:
        return f"Error: File not found at {file_path}"
    except json.JSONDecodeError as e:
        return f"Error decoding JSON: {e}"

# Example usage:
# file_path = 'path/to/your/persona.json'
# loaded_persona = load_persona_from_json(file_path)

# if loaded_persona:
#     print("Persona loaded successfully:")
#     print(json.dumps(loaded_persona, indent=2))
# else:
#     print("Failed to load persona.")


In [12]:
hank_persona = load_persona('hank_thompson.json')
claire_persona = load_persona('claire_reynolds.json')

# write out a regex
out = mistral + generate_daily_plan(hank_persona) # only executed after concatanating a prompt string

Exception: Attempted to use a context length of 512 tokens, but this LlamaCpp model is only configured to support up to 512!

In [42]:
out["tasks"]  # now we have a list of tasks

['Meet with the senior leadership team to discuss strategic plans, analyze market trends, and align company goals with industry growth opportunities',
 "Attend a board meeting to review financial reports, update the company's leadership, and address any issues or concerns from shareholders",
 'Host a networking event for employees and investors to foster relationships, promote team spirit, and generate new business ideas']

In [51]:
out4 = mistral + generate_start_times("CEO", out["tasks"])

In [43]:
out2 = mistral + estimate_duration("CEO", out["tasks"])
out2["duration"]

['3', '2', '4']

In [44]:
daily_plan = list(zip(out["tasks"], out2["duration"]))
daily_plan

[('Meet with the senior leadership team to discuss strategic plans, analyze market trends, and align company goals with industry growth opportunities',
  '3'),
 ("Attend a board meeting to review financial reports, update the company's leadership, and address any issues or concerns from shareholders",
  '2'),
 ('Host a networking event for employees and investors to foster relationships, promote team spirit, and generate new business ideas',
  '4')]

In [45]:
from datetime import datetime, timedelta

schedule = []
curr_time = datetime.now()

for item, duration in daily_plan:
    dur = timedelta(hours=int(duration))
    end_time = (curr_time + dur)
    print(f"{curr_time.strftime('%H:%M')} to {end_time.strftime('%H:%M')}: {item}")
    schedule.append({'start_time': curr_time, 'end_time': end_time, 'task': item})
    curr_time = end_time

11:44 to 14:44: Meet with the senior leadership team to discuss strategic plans, analyze market trends, and align company goals with industry growth opportunities
14:44 to 16:44: Attend a board meeting to review financial reports, update the company's leadership, and address any issues or concerns from shareholders
16:44 to 20:44: Host a networking event for employees and investors to foster relationships, promote team spirit, and generate new business ideas


In [48]:
schedule

[{'start_time': datetime.datetime(2024, 1, 3, 11, 44, 2, 803906),
  'end_time': datetime.datetime(2024, 1, 3, 14, 44, 2, 803906),
  'task': 'Meet with the senior leadership team to discuss strategic plans, analyze market trends, and align company goals with industry growth opportunities'},
 {'start_time': datetime.datetime(2024, 1, 3, 14, 44, 2, 803906),
  'end_time': datetime.datetime(2024, 1, 3, 16, 44, 2, 803906),
  'task': "Attend a board meeting to review financial reports, update the company's leadership, and address any issues or concerns from shareholders"},
 {'start_time': datetime.datetime(2024, 1, 3, 16, 44, 2, 803906),
  'end_time': datetime.datetime(2024, 1, 3, 20, 44, 2, 803906),
  'task': 'Host a networking event for employees and investors to foster relationships, promote team spirit, and generate new business ideas'}]

### Let's look at conversations

A conversation between two agents is triggered when an entity is detected. Relevant context about the entity is retrieved by each agent and then the agent is asked to react.

In [47]:
# @guidance
# def converse(lm, persona):
name = "Joe"
other = "Bob"
memory = {"Joe": ["likes bread", "does not like lights"], "Bob": ["Teehee, does not like bread"]}  # RAG pipeline for retrieval
location = "Bob entered the room carrying bread"

prompt = f"""
You are {name}. You are talking to {other}.

Observation: {observation}

Summary of {name}: {memory[name]}
Summary of relevant context about {other}: {memory[other]}

Example of dialogue:
A: Wow, it is a nice haircut
B: Thank you! How is your school project?
A: I'm still trying.
B: Good luck.

{name}'s reaction: {{reaction}}
What would {name} say to {other}? Make a short dialogue.

Here is the short dialogue:{gen('dialogue', max_tokens=1000)}"""

out3 = mistral + prompt

In [26]:
# persona + common strings
name1 = "Hank Thompson"
persona1 = "Hank Thompson, 54 years old, farmer, born and raised in a small town in Iowa, inherited family farm, continues the agricultural tradition with pride, enjoys tractor rides and classic country music, looks forward to local county fairs, likes hearty, home-cooked meals, dislikes city life, does not like genetically modified crops, pests and weeds"

name2 = "Claire Reynolds"
persona2 = "Claire Reynolds, 50 years old, agronomist, grew up in a suburban area, likes sustainable agriculture and rural life"

# location
location = "barn in Iowa"

# relationships
persona1topersona2 = "Hank admires Claire over their shared values"
persona2topersona1 = "Claire echoes Hank's aversion to city life, can relate to Hank's love for the countryside"

prompt = f"""
Generate a short dialogue between {name1} and {name2} in {location}

{name1} context: {persona1} {persona1topersona2}
{name2} context: {persona2} {persona2topersona1}

Example of dialogue:
Hank: Howdy, Claire, how's it going?
Claire: Good, what about you?

Here is the short dialogue:
{gen('dialogue', max_tokens=1000)}"""

out3 = mistral + prompt

Exception: Attempted to use a context length of 512 tokens, but this LlamaCpp model is only configured to support up to 512!

In [22]:
conversation = out3['dialogue']


Hank: Well, I'm just enjoying a little tractor ride around the farm.
Claire: That sounds lovely. I love spending time outdoors, especially when it comes to sustainable agriculture.
Hank: Absolutely, me too. I'm not a fan of those big city ways.
Claire: I couldn't agree more. There's something so peaceful and calming about being out in the countryside.
Hank: You know, I've been thinking about trying some of those new genetically modified crops, but I'm not sure I'm sold on the idea.
Claire: I hear you. I prefer to stick with traditional methods and natural solutions.
Hank: That's what I thought. We might have a lot in common, Claire.
Claire: I think so too, Hank. It's great to meet someone who shares my values and love for the land.


### Findings
1. Common string is neccessary. The persona needs to be concise as possible to preserve the context window
"bob is a blah, bob likes blah, bob dosen't like blah" <- common string

2. Not possible to create super long conversations with `llama-cpp`. Need to figure out how to do the server ASAP. `Nitro` looks promising

3. embeddings are essential to not violate the context window while still having realistic conversations. 